## Generate expected results for back tests

This notebook generates results for backtests. These include

* Backtests for run of simulation with default parameters (`test_backtest.py`).
* Backtests for runs of simulation where you are determining an appropriate number of replications (`test_backtest_replications.py`).

They save the relevant results dataframe as `.csv`, and use these to verify that the model produces consistent results.

The run time is provided at the end of this notebook.

## Set-up

Load required packages.

In [1]:
# pylint: disable=missing-module-docstring
# To ensure any updates to `simulation/` are fetched without needing to restart
# the notebook environment, reload `simulation/` before execution of each cell
%load_ext autoreload
%autoreload 1
%aimport simulation

In [2]:
# pylint: disable=wrong-import-position
# Import required packages
import os
import time
from IPython.display import display

from simulation.model import Param, Runner
from simulation.replications import (
    ReplicationsAlgorithm, ReplicationTabulizer, confidence_interval_method,
    confidence_interval_method_simple)

Start timer.

In [3]:
# Start timer
start_time = time.time()

Define path to expected results.

In [4]:
# Define path to folder for expected results for tests
TESTS = '../tests/exp_results/'

## Run of simulation with default parameters

In [5]:
# Define model parameters
param = Param(
    patient_inter=4,
    mean_n_consult_time = 10,
    number_of_nurses = 4,
    warm_up_period = 500,
    data_collection_period = 1500,
    number_of_runs=5,
    audit_interval = 50,
    scenario_name = 0,
    cores = 1
)

# Run the replications
experiment = Runner(param)
experiment.run_reps()

In [6]:
# Patient-level results
display(experiment.patient_results_df)
experiment.patient_results_df.to_csv(
    os.path.join(TESTS, 'patient.csv'), index=False)

,patient_id,arrival_time,q_time_nurse,time_with_nurse,run,q_time_unseen
0,1,504.513444,0.0,1.664896,0,NaN
1,2,507.230909,0.0,29.462508,0,NaN
2,3,508.394374,0.0,7.345594,0,NaN
3,4,512.975372,0.0,7.986073,0,NaN
4,5,521.933116,0.0,7.127312,0,NaN
...,...,...,...,...,...,...
1885,365,1978.104484,0.0,1.712059,4,NaN
1886,366,1980.171724,0.0,11.023619,4,NaN
1887,367,1986.506388,0.0,8.212316,4,NaN
1888,368,1988.648731,0.0,24.898160,4,NaN


In [7]:
# Run results
display(experiment.run_results_df)
experiment.run_results_df.to_csv(
    os.path.join(TESTS, 'run.csv'), index=False)

,run_number,scenario,arrivals,mean_q_time_nurse,mean_time_with_nurse,mean_nurse_utilisation,mean_nurse_utilisation_tw,mean_nurse_q_length,count_unseen,mean_q_time_unseen
0,0,0,403,2.043788,9.577436,0.640936,0.642332,0.549098,0,NaN
1,1,0,382,1.255117,10.208371,0.644744,0.646797,0.319636,0,NaN
2,2,0,369,3.392586,10.731510,0.662446,0.668010,0.835793,0,NaN
3,3,0,367,2.227175,10.523889,0.634062,0.639573,0.544915,0,NaN
4,4,0,369,1.653496,9.809751,0.601042,0.601042,0.406760,0,NaN


In [8]:
# Interval audit results
display(experiment.interval_audit_df)
experiment.interval_audit_df.to_csv(
    os.path.join(TESTS, 'interval.csv'), index=False)

,resource_name,simulation_time,utilisation,queue_length,running_mean_wait_time,run
0,nurse,500,0.50,0,0.258217,0
1,nurse,550,0.00,0,0.237723,0
2,nurse,600,0.00,0,0.220244,0
3,nurse,650,1.00,0,0.215357,0
4,nurse,700,1.00,1,1.677251,0
...,...,...,...,...,...,...
145,nurse,1750,1.00,0,2.228919,4
146,nurse,1800,0.50,0,2.174401,4
147,nurse,1850,1.00,1,2.116596,4
148,nurse,1900,0.75,0,2.121569,4


In [9]:
# Overall results
display(experiment.overall_results_df)
experiment.overall_results_df.to_csv(
    os.path.join(TESTS, 'overall.csv'), index=True)

,arrivals,mean_q_time_nurse,mean_time_with_nurse,mean_nurse_utilisation,mean_nurse_utilisation_tw,mean_nurse_q_length,count_unseen,mean_q_time_unseen
mean,378.000000,2.114432,10.170191,0.636646,0.639551,0.531240,0.0,NaN
std_dev,15.198684,0.806577,0.480342,0.022494,0.024255,0.195914,0.0,NaN
lower_95_ci,359.128341,1.112935,9.573768,0.608716,0.609435,0.287981,0.0,NaN
upper_95_ci,396.871659,3.115930,10.766615,0.664576,0.669667,0.774500,0.0,NaN


## Running the simulation when attempting to determine an appropriate number of parameters

Each of these should return the same output dataframe:

```
_, man_df_simple = confidence_interval_method_simple(
    replications=20, metric='mean_time_with_nurse')
```

```
_, man_df = confidence_interval_method(
    replications=20, metric='mean_time_with_nurse')
```

```
observer = ReplicationTabulizer()
analyser = ReplicationsAlgorithm(
    verbose=False,
    observer=observer,
    initial_replications=20,
    replication_budget=20)
_ = analyser.select(runner=Runner(Param()), metric='mean_time_with_nurse')
auto_df = observer.summary_table().head(20)
```

Hence, we will just run one, but will compare all against it in backtests.

In [10]:
_, man_df = confidence_interval_method(
    replications=20, metric='mean_time_with_nurse')

display(man_df)

man_df.to_csv(
    os.path.join(TESTS, 'replications.csv'), index=False)

Reached desired precision (0.05) in 5 replications.


,replications,data,cumulative_mean,stdev,lower_ci,upper_ci,deviation
0,1,9.842380,9.842380,NaN,NaN,NaN,NaN
1,2,10.060424,9.951402,NaN,NaN,NaN,NaN
2,3,9.925090,9.942631,0.110075,9.669189,10.216073,0.027502
3,4,9.938504,9.941599,0.089900,9.798549,10.084650,0.014389
4,5,10.016611,9.956602,0.084775,9.851339,10.061864,0.010572
5,6,9.883801,9.944468,0.081442,9.859000,10.029936,0.008595
6,7,10.040555,9.958195,0.082742,9.881671,10.034719,0.007685
7,8,10.086612,9.974247,0.089048,9.899801,10.048693,0.007464
8,9,10.202806,9.999643,0.112884,9.912872,10.086413,0.008677
9,10,10.093238,10.009002,0.110467,9.929979,10.088025,0.007895


## Run time

In [11]:
# Get run time in seconds
end_time = time.time()
runtime = round(end_time - start_time)

# Display converted to minutes and seconds
print(f'Notebook run time: {runtime // 60}m {runtime % 60}s')

Notebook run time: 0m 2s
